In [201]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix , mean_squared_error

df= pd.read_csv('/Users/burakfindikli/Documents/repos/ML_Analiz/data/2018-2019_tsl.csv',index_col = False)
df2=pd.read_csv('/Users/burakfindikli/Documents/repos/ML_Analiz/data/2018-2019_tsl.csv',index_col = False)

In [202]:
 #  2017-2018 ile 2018-2019 sezonunun Maçlarını birleştirdim
 df3=pd.concat([df, df2], ignore_index=True)


In [203]:
#Maçı ev sahibi kazandıysa 1 beraberlik olduysa 0 deplasman takımı kazandıysa 2 diye kodladım.
df3["kazanan_dep"]=df3.ev_sonuc < df3.dep_sonuc
df3.kazanan_dep = df3.kazanan_dep.astype(int)*2

df3["kazanan_ev"] = df3.ev_sonuc > df3.dep_sonuc
df3.kazanan_ev = df3.kazanan_ev.astype(int)

df3["kazanan_berabere"] = df3.ev_sonuc == df3.dep_sonuc
df3.kazanan_berabere = df3.kazanan_berabere.astype(int)*0

df3["sonuc"]=df3.kazanan_dep+df3.kazanan_ev+df3.kazanan_berabere
df3=df3.drop(['kazanan_dep', 'kazanan_ev','kazanan_berabere','ev_gol','dep_gol'], axis=1)

In [204]:
#bu değişkenlerde % ve , var temizleme yapmam lazım
col=['ev_topla_oynama','ev_ikili_mucadele_kazanma','ev_hava_topu','dep_topla_oynama','dep_ikili_mucadele_kazanma','dep_hava_topu','ev_pas_isabet','ev_rakip_yari_sahada_pas_isabeti','ev_orta_istabeti','dep_pas_isabet','dep_rakip_yari_sahada_pas_isabeti','dep_orta_istabeti','ev_sut_isabeti','dep_sut_isabeti','ev_top_kapma_basarisi','dep_top_kapma_basarisi']

In [205]:
# col değişkenindeki % , gibi karakterleri replace ettim
k=0
while k < df3[col].shape[1]:
    df3[col[k]] = df3[col[k]].str.replace(",",".")
    df3[col[k]] = df3[col[k]].str.replace("%","")
    k+=1

In [206]:
#object olan veri tiplerini floata çevirdim
df3[col]=df3[col].astype(float)

In [207]:
drop_value = ['ev_takim', 'dep_takim', 'ev_sonuc', 'dep_sonuc', ]
df3=df3.drop(drop_value,axis=1)

In [208]:
from sklearn.model_selection import train_test_split , GridSearchCV

In [209]:
#bağımlı bağımsız değişkenlerimi ayırdım 
#bağımlı değişken sonuc(1=Evsahibi kazanır 0=Beraberlik 2=Deplasman kazanır)
x = df3.iloc[:,:-1]
y = df3.sonuc
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
x_scaled = stdsc.fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split(x_scaled,y,test_size=0.2,random_state=101)

In [210]:
from sklearn.svm import SVC
model=SVC(kernel="poly").fit(X_train, y_train)
model

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [211]:
y_pred = model.predict(X_test)

In [212]:
model.score(X_test,y_test) 
#mean_squared_error(y_test,y_pred)

0.8151260504201681

In [213]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[ 70  49   2]
 [  6 208   4]
 [  1  26 110]]


In [214]:
model=SVC(kernel="poly")
svm_params = {"C" : [0.1,0.3,0.5,1,2,3,4,5],"kernel" : ['linear', 'poly', 'rbf']}
svm_cv_model = GridSearchCV(model,svm_params,cv=10,verbose=2,n_jobs=-1).fit(X_train,y_train)

Fitting 10 folds for each of 24 candidates, totalling 240 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   42.2s
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  1.6min finished


In [215]:
svm_cv_model.best_params_

{'C': 5, 'kernel': 'rbf'}

In [219]:
svm_tuned =SVC(kernel=svm_cv_model.best_params_["kernel"], C=svm_cv_model.best_params_["C"]).fit(X_train,y_train)
y_pred= svm_tuned.predict(X_test)

In [220]:
mean_squared_error(y_test,y_pred)

0.20798319327731093

In [221]:
svm_tuned.score(X_test,y_test) 

0.8928571428571429